In [2]:
import cv2
import mediapipe as mp
import math
import numpy as np

mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils

hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.8,
    min_tracking_confidence=0.8
)

In [3]:
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - \
              np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180:
        angle = 360 - angle
    return angle

In [ ]:
# ---------- Distance helper ----------
dist = lambda a,b: math.hypot(a[0]-b[0], a[1]-b[1])

# ---------- Capturing video ----------
cap = cv2.VideoCapture(0)

# paused = False #new#

while True:

    # if not paused: #new#
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    res = hands.process(rgb)
    label = ""

    if res.multi_hand_landmarks and res.multi_handedness:
        for hand_lm, handed in zip(res.multi_hand_landmarks,
                                   res.multi_handedness):

            if handed.classification[0].label != "Left":
                continue

            # pixel landmarks
            lm = [(int(p.x*w), int(p.y*h)) for p in hand_lm.landmark]

            # normalized landmarks (for angle)
            nlm = hand_lm.landmark
            a = [nlm[mp_hands.HandLandmark.THUMB_CMC].x,
                 nlm[mp_hands.HandLandmark.THUMB_CMC].y]
            b = [nlm[mp_hands.HandLandmark.THUMB_MCP].x,
                 nlm[mp_hands.HandLandmark.THUMB_MCP].y]
            c = [nlm[mp_hands.HandLandmark.INDEX_FINGER_TIP].x,
                 nlm[mp_hands.HandLandmark.INDEX_FINGER_TIP].y]

            angle = calculate_angle(a, b, c)

            # palm scale
            p = dist(lm[0], lm[9])

            # ---------- fingertip distances ----------
            thumb_tip = dist(lm[4], lm[0]) / p
            index_tip = dist(lm[8], lm[0]) / p
            middle_tip = dist(lm[12], lm[0]) / p
            ring_tip = dist(lm[16], lm[0]) / p
            pinky_tip = dist(lm[20], lm[0]) / p
            
            cv2.putText(frame, f"thumb:{thumb_tip:.2f}", (10, 150),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.putText(frame, f"index:{index_tip:.2f}", (10, 180),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.putText(frame, f"mid:{middle_tip:.2f}", (10, 210),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.putText(frame, f"ring:{ring_tip:.2f}", (10, 240),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.putText(frame, f"pinky:{pinky_tip:.2f}", (10, 270),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)


            # ---------- KEITO DETECTION ----------
            keito = (
                dist(lm[4], lm[8]) / p < 1.00 and
                dist(lm[12], lm[0]) / p > 1.45 and
                dist(lm[16], lm[0]) / p > 1.40 and
                dist(lm[20], lm[0]) / p > 1.30 and
                100 < angle < 111 and 
                lm[8][0] < lm[2][0] and lm[8][1] > lm[2][1]    
            )
            # ---------- KOKO WAZA DETECTION ----------
            koko = (
                dist(lm[4], lm[0]) / p < 1.34 and
                dist(lm[8], lm[0]) / p > 1.90 and
                dist(lm[12], lm[0]) / p > 1.85 and
                dist(lm[16], lm[0]) / p > 1.80 and
                dist(lm[20], lm[0]) / p > 1.60 and
                115 < angle < 125 and 
                lm[8][0] < lm[4][0] and lm[8][1] < lm[4][1]    
            )
             # ---------- IPPON NUKITE DETECTION ----------
            ippon = (
                dist(lm[4], lm[0])/p < 1.15  and 
                dist(lm[8], lm[0])/p > 1.75  and  
                dist(lm[12], lm[0])/p < 1.15 and  
                dist(lm[16], lm[0])/p < 1.15 and 
                dist(lm[20], lm[0])/p < 1.15 and  
                110 < angle < 125 and 
                lm[8][0] < lm[12][0] and lm[12][1] > lm[4][1]
            )
             # ---------- NIHON NUKITE DETECTION ----------
            nipon = (
                dist(lm[4], lm[0])/p < 1.10  and 
                dist(lm[8], lm[0])/p > 1.75  and  
                dist(lm[12], lm[0])/p > 1.70 and  
                dist(lm[16], lm[0])/p < 1.05 and  
                dist(lm[20], lm[0])/p < 1.15 and   
                lm[12][0] < lm[16][0] and lm[12][1] > lm[4][1]
            )
            # ---------- SHIHON NUKITE DETECTION ----------
            shihon = (
                dist(lm[4], lm[0])/p < 0.95  and  
                dist(lm[8], lm[0])/p > 1.70  and  
                dist(lm[12], lm[0])/p > 1.80 and  
                dist(lm[16], lm[0])/p > 1.70 and  
                dist(lm[20], lm[0])/p > 1.40 and  
                105 < angle < 120 and 
                lm[8][0] < lm[4][0] and lm[12][1] > lm[4][1]
            )
            # ---------- WASHIDE DETECTION ----------
            washide = (
                dist(lm[4], lm[0])/p < 1.40  and  
                dist(lm[8], lm[0])/p > 1.30  and  
                dist(lm[12], lm[0])/p > 1.40 and  
                dist(lm[16], lm[0])/p > 1.30 and
                dist(lm[20], lm[0])/p > 1.20 and 
                150 < angle < 160 and 
                lm[5][0] < lm[4][0] and lm[5][1] < lm[4][1]
            )
            # ---------- NAKADAKA DETECTION ----------
            nakadaka = (
                dist(lm[4], lm[0])/p < 1.20  and  
                dist(lm[8], lm[0])/p > 0.80  and 
                dist(lm[12], lm[0])/p > 0.80 and  
                dist(lm[16], lm[0])/p > 0.60 and  
                dist(lm[20], lm[0])/p > 0.60 and  
                70 < angle < 100 and
                lm[4][0] > lm[10][0] and lm[8][1] < lm[4][1]
            )

            if keito:    
                label = "KEITO(Bent wrist)"
            elif koko:
                label = "KOKO WAZA(Tiger-mouth strike)"
            elif ippon:
                label = "IPPON NUKITE(One-finger spear)"
            elif nipon:
                label = "NIPON NUKITE(Two-finger spear)"
            elif shihon:
                label = "SHIHON NUKITE(Four-finger spear hand)"
            elif washide:
                label = "WASHIDE(Eagle-claw hand)"
            elif nakadaka:
                label = "NAKADAKA(Middle-knuckle strike)"
            else:
                label = "Adjust the Position"

            mp_draw.draw_landmarks(frame, hand_lm,mp_hands.HAND_CONNECTIONS)
            break

    if label:
        cv2.putText(frame, label, (30,50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255,0,0), 3)
        cv2.putText(frame, f"Angle: {int(angle)}",
                    (30,95),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,0), 2)

    cv2.imshow("Karate Hand Symbol Recognition", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()